In [1]:
import pybitflyer
import json
import time
from threading import Thread
import threading
import os
import pandas as pd
import talib
import matplotlib.pyplot as plt
import datetime
from IPython.display import display, clear_output
import requests
import numpy as np
import xgboost 
from sklearn.model_selection import train_test_split, GridSearchCV
import copy
import datetime
import pickle
import utils
import xgboost_ohlc
%matplotlib inline

In [2]:
# -------------------------------------

end_point = 'wss://ws.lightstream.bitflyer.com/json-rpc'

public_channels = ['lightning_executions_FX_BTC_JPY','lightning_board_snapshot_FX_BTC_JPY']
private_channels = ['child_order_events', 'parent_order_events']
# -------------------------------------

In [3]:
def trade_loop(pyb):
    ohlc = pd.read_csv('./scraping/5min.csv')
    ohlc = ohlc.dropna()
    ohlc, data_columns, target_columns = utils.shaping_ohlc(ohlc)
    X_train = ohlc[data_columns]
    y_r_train = ohlc['TC/C']
    y_c_train = ohlc['Class']

    xgbr = xgboost_ohlc.r_fit(X_train,y_r_train)
    xgbc = xgboost_ohlc.c_fit(X_train,y_c_train)

    base_volume = [0.01,0.02]
    bv_ind = 0
    while True:
        if 1 < time.time() % 300 and time.time() % 300 < 11:
            
            state = pyb.getboardstate(product_code='FX_BTC_JPY')
            if type(state) == dict and 'state' in state.keys() and state['state'] == 'RUNNING' and state['health'] in ['NORMAL','BUSY','VERY BUSY']:

                ohlc = utils.get_ohlc(periods=300,datasize=500)
                ohlc, data_columns, target_columns = utils.shaping_ohlc(ohlc,drop=False)

                X_test = ohlc[-1:][data_columns]


                flag = xgbr.predict(X_test)[0] >= 1
                flag2 = xgbc.predict(X_test)[0] == 1
                if (flag and flag2) or (not flag and not flag2):
                    bv_ind = 1
                else:
                    bv_ind = 0
                price = int(xgbr.predict(X_test)[0] * ohlc[-1:]['Close'])
                rate = xgbr.predict(X_test)[0]

                side = 'NO'
                position = pyb.getpositions(product_code="FX_BTC_JPY")
                pos_volume = 0
                if type(position) == list and len(position) >= 1:
                    side = position[0]['side']
                    for p in position:
                        pos_volume += p['size']


                board = pyb.board(product_code="FX_BTC_JPY")

                result = 'continue'
                if flag2 and side == 'SELL' and round(pos_volume+base_volume[bv_ind],2)>=0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="BUY",
                        size=round(pos_volume+base_volume[bv_ind],2),
                        price= board['bids'][0]['price']+1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif not flag2 and side == 'BUY' and round(pos_volume+base_volume[bv_ind],2)>=0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="SELL",
                        size=round(pos_volume+base_volume[bv_ind],2),
                        price=board['asks'][0]['price']-1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif flag2 and side == 'NO' and base_volume[bv_ind] >= 0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="BUY",
                        size=base_volume[bv_ind],
                        price=board['bids'][0]['price']+1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif not flag2 and side == 'NO' and base_volume[bv_ind] >= 0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="SELL",
                        size=base_volume[bv_ind],
                        price=board['asks'][0]['price']-1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                #買いまし 売りまし
                elif flag2 and side == 'BUY' and round(base_volume[bv_ind]-pos_volume,2)>=0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="BUY",
                        size=round(base_volume[bv_ind]-pos_volume,2),
                        price=board['bids'][0]['price']+1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif flag2 and side == 'BUY' and round(base_volume[bv_ind]-pos_volume,2)<=-0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="SELL",
                        size=round(pos_volume-base_volume[bv_ind],2),
                        price=board['bids'][0]['price']+1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif not flag2 and side == 'SELL' and round(base_volume[bv_ind]-pos_volume,2)>=0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="SELL",
                        size=round(base_volume[bv_ind]-pos_volume,2),
                        price=board['asks'][0]['price']-1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )
                elif not flag2 and side == 'SELL' and round(base_volume[bv_ind]-pos_volume,2)<=-0.01:
                    result = pyb.sendchildorder(
                        product_code="FX_BTC_JPY",
                        child_order_type="LIMIT",
                        side="BUY",
                        size=round(pos_volume-base_volume[bv_ind],2),
                        price=board['asks'][0]['price']-1,
                        minute_to_expire=4,
                        time_in_force="GTC"
                    )

                #xgbr,xgbc = xgb_predict(periods=300,datasize=5000)
                
                history = pyb.getcollateralhistory()
                if type(history) == list and len(history) >= 1:
                    history_count = 0
                    history_sum = 0
                    for h in history:
                        history_count += (h['change']>0)*1
                        history_sum += h['change']
                    if history_sum > 0 and history_count > 90:
                        base_volume = [0.04,0.1]
                    elif history_sum > 0 and history_count > 75:
                        base_volume = [0.03,0.7]
                    elif history_sum > 0 and history_count > 50:
                        base_volume = [0.02,0.03]
                    else:
                        base_volume = [0.01,0.02]
                        
                print(flag,flag2,price, result,rate)
                print(datetime.datetime.now())
                print('ohlc gap',time.time()-ohlc['timestamp'].values[-1])
                print(base_volume,bv_ind,pos_volume,history_count,history_sum)

                time.sleep(10)
            else:
                time.sleep(1)
        else:
            time.sleep(1)

In [4]:
if __name__ == '__main__':
    
    pyb = pybitflyer.API(api_key=key,api_secret=secret)
    
    tradeThread = Thread(target=trade_loop,args=(pyb,))
    tradeThread.start()

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   28.3s finished


Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   58.2s finished


True True 1230567 {'child_order_acceptance_id': 'JRF20200826-025504-087286'} 1.0000918
2020-08-26 11:55:04.814211
ohlc gap 4.814260959625244
[0.01, 0.02] 1 0 56 -137.0
False True 1231283 {'child_order_acceptance_id': 'JRF20200826-030004-080755'} 0.99996847
2020-08-26 12:00:04.601911
ohlc gap 4.601948022842407
[0.01, 0.02] 0 0.02 56 -137.0
True True 1228563 {'child_order_acceptance_id': 'JRF20200826-030504-032933'} 1.0000973
2020-08-26 12:05:04.822557
ohlc gap 4.82259202003479
[0.01, 0.02] 1 0.01 56 -162.0
True True 1228283 continue 1.0001482
2020-08-26 12:10:04.610520
ohlc gap 4.610559940338135
[0.01, 0.02] 1 0.02 56 -162.0
True True 1227709 continue 1.0001482
2020-08-26 12:15:04.462577
ohlc gap 4.462613105773926
[0.01, 0.02] 1 0.02 56 -162.0
False True 1227965 {'child_order_acceptance_id': 'JRF20200826-032004-069087'} 0.9999995
2020-08-26 12:20:05.535193
ohlc gap 5.535268068313599
[0.01, 0.02] 0 0.02 55 -217.0
True True 1227239 {'child_order_acceptance_id': 'JRF20200826-032503-118871'

In [ ]:
pickle.dump(xgbr, open("xgbc_model_demo.pickle", "wb"))
pickle.dump(xgbc, open("xgbc_model_demo.pickle", "wb"))

In [6]:
ppp = pybitflyer.API(api_key=key,api_secret=secret)

In [6]:
pos = ppp.getpositions(product_code="FX_BTC_JPY")

In [7]:
col = ppp.getcollateralhistory()

In [8]:
col

[{'id': 3461,
  'currency_code': 'JPY',
  'change': -15.0,
  'amount': 53628.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T16:10:04.217'},
 {'id': 3460,
  'currency_code': 'JPY',
  'change': 7.0,
  'amount': 53643.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T15:55:50.05'},
 {'id': 3459,
  'currency_code': 'JPY',
  'change': -19.0,
  'amount': 53636.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T15:50:04.54'},
 {'id': 3458,
  'currency_code': 'JPY',
  'change': -24.0,
  'amount': 53655.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T15:45:06.383'},
 {'id': 3457,
  'currency_code': 'JPY',
  'change': 6.0,
  'amount': 53679.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T15:30:04.07'},
 {'id': 3456,
  'currency_code': 'JPY',
  'change': -22.0,
  'amount': 53673.0,
  'reason_code': 'CLEARING_COLL',
  'date': '2020-08-12T15:15:47.247'},
 {'id': 3455,
  'currency_code': 'JPY',
  'change': -21.0,
  'amount': 53695.0,
  'reason_code'

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    7.1s finished


True 1247868 continue 1.0007272
2020-08-13 01:15:16.810569
ohlc gap 16.81061601638794
0.01 49 -1809.0


In [8]:
type(pos)

list

False 1277051 continue 0.9992237
2020-08-10 15:25:04.207831
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    7.8s finished


In [ ]:
board = ppp.getboardstate(product_code="FX_BTC_JPY")

In [ ]:
board = ppp.board(product_code="FX_BTC_JPY")

In [7]:
result = ppp.sendparentorder(
    order_method =  "SIMPLE",
    minute_to_expire = 5,
    time_in_force = "GTC",
    parameters = [{
      "product_code": "FX_BTC_JPY",
      "condition_type": "LIMIT",
      "side": "BUY",
      "size": 0.01,
      "price": 1100000
    }]
)

In [9]:
result = ppp.sendchildorder(
                   product_code="FX_BTC_JPY",
                   child_order_type="LIMIT",
                   side="BUY",
                   size=0.01,
                   price=1100000,
                   minute_to_expire=4,
                   time_in_force="GTC"
                   )

In [10]:
result

{'child_order_acceptance_id': 'JRF20200803-055904-014209'}

In [8]:
result

{'status': -101, 'error_message': 'Invalid type of order', 'data': None}

In [9]:
result

{'parent_order_acceptance_id': 'JRF20200802-160806-002716'}

In [31]:
rrr = ppp.getpositions(product_code="FX_BTC_JPY")

In [39]:
rrr[0]['side']

'BUY'

In [33]:
rrr[0]

{'product_code': 'FX_BTC_JPY',
 'side': 'BUY',
 'price': 1206988.0,
 'size': 0.01,
 'commission': 0.0,
 'swap_point_accumulate': 0.0,
 'require_collateral': 3017.47,
 'open_date': '2020-08-02T11:12:44.277',
 'leverage': 4.0,
 'pnl': -4.41,
 'sfd': 0.0}

In [1]:
d = {}
d.keys()

dict_keys([])

In [13]:
[1,2,3,4][:100]

[1, 2, 3, 4]

In [11]:
not True and not True and not True

False

True False 1279626 {'child_order_acceptance_id': 'JRF20200815-094503-146115'} 1.0000796
2020-08-15 18:45:03.393611
ohlc gap 3.3936471939086914
[0.0, 0.03] 0 0.03 55 -14.0
True False 1278996 continue 1.0000796
2020-08-15 18:50:02.813932
ohlc gap 2.8139710426330566
[0.0, 0.03] 0 0 55 -31.0
